In [1]:
"""
Created on Fri Feb 19 08:46:16 2021

@author: Schroeder
"""

# Liest holc etc. daten von bereits auf platte befindlichen csv dataframes .
# Soll Plot und Auswertung von datagrabbing trennen.

import plotly.io as pio
import plotly.express as px
from plotly.figure_factory import create_candlestick
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

#import TickerSelector, regression
from datetime import datetime
from datetime import date
import calendar

# Imports
from pandas_datareader import data as pdr
#from yahoo_fin import stock_info as si

import numpy as np
import scipy.stats as stats


from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
import csv
   
#import StockscreenerPlotter_stats
#import StockscreenerWinners_stats
#import downloader

import sys
import os.path
from os.path import isfile, join

from os import listdir
#from os.path import isfile, join

#currentdir = os.path.join(os.path.dirname(__file__))

###### Insert the following three lines to make any import lib in he project dir setup visible to an other
###### Directory in the project setup

currentdir = os.path.abspath('')
parentdir = os.path.realpath(os.path.join(currentdir, '..'))
sys.path.insert(0, parentdir) 
#############################################################

from lib import Indikatoren

#lieferrt bei gegebenem datatfram mit datum als index das erste Datum, das letzte, und zwei 
# dazwischen


    

## SCANNER sucht in Data Universum (etwa SP500 oder Aristokraten) nach Stocks, die ein bestimmtes Signal aufweisen. Also etwa 
## weekly MACD positiv und RSI > 75% etc.

## Gebe Pfad an, unter dem das Datenfile zu finden ist.
## Bsp: C:/Users/Schroeder/OneDrive/Trading/ETFS/RES_Aristokraten/Res/


# --------------------------------------------------------------------------
## Welcher Sektor oder index soll analysieert werden ?

In [2]:
pfad = os.path.realpath(os.path.join(parentdir, 'ETFS'))

#############################################################
who = "RES_EUsektoren"
pfad = os.path.realpath(os.path.join(pfad, who))
###############################################################
pfad = os.path.realpath(os.path.join(pfad, 'Data'))


file  = "_ticker_names.csv"

singleticker_file=os.path.realpath(os.path.join(pfad, file))



In [3]:
exportList = pd.DataFrame(columns=['ticker',"z-score","Volume","MACDsignal","RSIlevel","Name","Industry","Sector","marketCap"])

## ---------------------------------------------------------------------

In [4]:
# DAs ist der Ticker fiel, der alle Ticker enthaelt, die gelsen werden:
tkr = pd.read_csv(singleticker_file,sep=";",decimal=',',parse_dates=True,index_col=0)
        #df = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    
for i in tkr["ticker"]:
    mydict={}
    file=pfad+ "/"  + i + ".csv"
    df = pd.read_csv(file,sep=";",decimal=',',parse_dates=True,index_col=0)
    #print(file)
    df["dummy"]= df["Factor"]-df["EMA100"]
    df["dummy"]=df["dummy"].fillna(0)
    Indikatoren.Zscore_rolling(df,"dummy",100)
    #df["Zscore"]=stats.zscore(df["dummy"])
    #Prüfe auf z-score
    #########################################################################
    if (df["z-score"][-1] < -2.5) or (df["z-score"][-1] >2.5):
    ########################################################################
        mydict["ticker"]=i
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
        mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
        mydict["z-score"]=df["z-score"][-1]
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        
        
    if((df["MACDdif_week"][-1] > 0) and \
        ((df["MACD_week"][-1] > 0) and (df["MACDsign_week"][-1] < 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Long"
    if((df["MACDdif_week"][-1] < 0) and \
        ((df["MACD_week"][-1] < 0) and (df["MACDsign_week"][-1] > 0))):
            mydict["ticker"]=i
            mydict["Name"]=tkr["name"][tkr["ticker"]==i]
            mydict["Sector"]=tkr["sector"][tkr["ticker"]==i]
            mydict["Industry"]=tkr["industry"][tkr["ticker"]==i]
            mydict["MACDsignal"]="Short"        
    
    
    if i == "HRB":
        fig = px.line(df, x=df.index, y=[df["MACD_week"],df["MACDsign_week"]])
        fig.show()  

    if len(mydict)>0:
        exportList =exportList.append(mydict,ignore_index=True)  
#exportList.style.background_gradient()
print(exportList.sort_values(by="z-score",ascending=False))


    ticker   z-score  Volume MACDsignal  RSIlevel  \
113   MODN  3.805338     NaN        NaN       NaN   
10     AIT  3.767560     NaN       Long       NaN   
182    VPG  3.740971     NaN        NaN       NaN   
173    THR  3.581950     NaN        NaN       NaN   
183    VVI  3.482582     NaN        NaN       NaN   
..     ...       ...     ...        ...       ...   
187    WLK       NaN     NaN      Short       NaN   
188     WM       NaN     NaN       Long       NaN   
191    WNC       NaN     NaN       Long       NaN   
192    WSR       NaN     NaN      Short       NaN   
194   WTTR       NaN     NaN      Short       NaN   

                                                  Name  \
113     835    Model N, Inc.
Name: name, dtype: object   
10   238    Applied Industrial Technologies, Inc.
N...   
182  1224    Vishay Precision Group, Inc.
Name: nam...   
173  1141    Thermon Group Holdings, Inc.
Name: nam...   
183        1219    Viad Corp
Name: name, dtype: object   
..             